In [37]:
import numpy as np
import matplotlib.pyplot as plt
from ansys.mapdl.core import launch_mapdl
mapdl=launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise Academic Student
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [38]:
# Takes in Pressure, Crack length and Thickness
def calculateK (press,a,t):
    mapdl.clear()
    mapdl.prep7()
    
    # Define Material Property
    E = 210e9
    nu = 0.3
    YS = 240e6
    # Defining Geometry 
    # Corners of the rectangle
    m0 = mapdl.k("", 0, -0.1)
    m1 = mapdl.k("", 0,0.1)
    m4 = mapdl.k("", 0.1, 0.1)
    m5 = mapdl.k("", 0.1, -0.1)
    main = mapdl.a(m0,m1,m4,m5)
    # Crack geometry
    k0 = mapdl.k("",0,-0.0001)
    k1 = mapdl.k("",0.9*a,-0.0001)
    k2 = mapdl.k("",a,0)
    k3 = mapdl.k("",0.9*a,0.0001)
    k4 = mapdl.k("",0,0.0001)
    crack = mapdl.a(k0, k1, k2,k3,k4)
    # Substract the areas and extrude with Thickness t
    aout = mapdl.asba(main, crack)
    mapdl.vext(aout, dz=t)
    #mapdl.vplot(show_lines=True, quality=5)
    mapdl.et(1,"SOLID285")
    # Define the Mesh
    mapdl.smrtsize(4)
    mapdl.aesize("ALL", 0.0035)
    mapdl.vmesh(1)

    
    #mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")
    
    # Using the SI units
    mapdl.units("SI")  # SI - International system (m, kg, s, K).

    # Define a material (nominal steel in SI)
    mapdl.mp("EX", 1, E)  # Elastic moduli in Pa (kg/(m*s**2))
    mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
    mapdl.mp("NUXY", 1, nu)  # Poisson's Ratio

    # Fix the lower side.
    mapdl.nsel("S", "LOC", "Y", -0.1)
    mapdl.d("ALL", "UY")
    mapdl.d("ALL", "UZ")
    mapdl.d("ALL", "UX")

    mapdl.nsel("S", "LOC", "Y", 0.1)
    mapdl.cp(5, "UX", "ALL")

    mapdl.asel("S",'LOC',"Y",0.1)
    mapdl.sfa('ALL', '', 'PRES', -press)
    mapdl.allsel(mute=True)
    mapdl.run("/SOLU")
    mapdl.antype("STATIC")
    mapdl.solve()
    mapdl.finish(mute=True)
    mapdl.post1()
    mapdl.path('P',3,5)
    node1 = mapdl.queries.node(a,0,0)
    node2 = mapdl.queries.node(0.5*a,0.0001,0)
    node3 = mapdl.queries.node(0,0.0001,0)
    mapdl.ppath(1, node1)
    mapdl.ppath(2, node2)
    mapdl.ppath(3, node3)
    mapdl.local(11,0,a,0,0)
    mapdl.csys(11)
    a = mapdl.kcalc(0,1,0,0)
    result = mapdl.result
    #result.plot_principal_nodal_stress(
    #    0,
    #    "SEQV",
    #    lighting=False,
    #    background="w",
    #    show_edges=True,
    #    text_color="k",
    #    add_text=False,
    #    )
    words = a.split(' ')
    
    # Extracting the K1 value from the String output
    for i in range(len(words)):
        if words[i]=='KI':
            x=i
    K1 = words[x+3][:-1] 
    print(f"K1 is equal to {K1}")
    return K1

In [39]:
K1 = calculateK(200e5,0.012,0.012)

K1 is equal to 0.43408E+07


In [40]:
# Variables used P: distribute load   A: Crack length    T:  Thickness
P = [100e5,150e5,200e5,300e5,350e5,400e5,450e5,500e5,550e5,600e5,650e5,700e5,750e5,800e5,850e5,900e5,950e5,1000e5] 
A = [0.010, 0.011,0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.020]
T = [0.006,0.007,0.008,0.009,0.010, 0.011,0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.020]
data = np.zeros((len(P)*len(A)*len(T),4))
count =0
for i in range(len(P)):
    for j in range(len(A)):
        for k in range(len(T)):
            print(count)
            K1 = calculateK(P[i],A[j],T[k])
            data[count,0] = P[i]
            data[count,1] = A[j]
            data[count,2] = T[k]
            data[count,3] = K1
            count=count+1    
      
            
        

0
K1 is equal to 0.18403E+07
1
K1 is equal to 0.17859E+07
2
K1 is equal to 0.16849E+07
3
K1 is equal to 0.17395E+07
4
K1 is equal to 0.18034E+07
5
K1 is equal to 0.17332E+07
6
K1 is equal to 0.18076E+07
7
K1 is equal to 0.17507E+07
8
K1 is equal to 0.17036E+07
9
K1 is equal to 0.16806E+07
10
K1 is equal to 0.16257E+07
11
K1 is equal to 0.18007E+07
12
K1 is equal to 0.17437E+07
13
K1 is equal to 0.16719E+07
14
K1 is equal to 0.17845E+07
15
K1 is equal to 0.19184E+07
16
K1 is equal to 0.20247E+07
17
K1 is equal to 0.18717E+07
18
K1 is equal to 0.20290E+07
19
K1 is equal to 0.18858E+07
20
K1 is equal to 0.19203E+07
21
K1 is equal to 0.19287E+07
22
K1 is equal to 0.20287E+07
23
K1 is equal to 0.18447E+07
24
K1 is equal to 0.19106E+07
25
K1 is equal to 0.19179E+07
26
K1 is equal to 0.18952E+07
27
K1 is equal to 0.17526E+07
28
K1 is equal to 0.17732E+07
29
K1 is equal to 0.19574E+07
30
K1 is equal to 0.21657E+07
31
K1 is equal to 0.22044E+07
32
K1 is equal to 0.22490E+07
33
K1 is equal to 0.

K1 is equal to 0.40286E+07
269
K1 is equal to 0.40422E+07
270
K1 is equal to 0.43343E+07
271
K1 is equal to 0.43359E+07
272
K1 is equal to 0.44378E+07
273
K1 is equal to 0.43329E+07
274
K1 is equal to 0.43141E+07
275
K1 is equal to 0.43633E+07
276
K1 is equal to 0.44488E+07
277
K1 is equal to 0.42197E+07
278
K1 is equal to 0.43381E+07
279
K1 is equal to 0.43547E+07
280
K1 is equal to 0.41941E+07
281
K1 is equal to 0.42447E+07
282
K1 is equal to 0.42854E+07
283
K1 is equal to 0.43214E+07
284
K1 is equal to 0.44113E+07
285
K1 is equal to 0.45492E+07
286
K1 is equal to 0.44419E+07
287
K1 is equal to 0.45754E+07
288
K1 is equal to 0.45485E+07
289
K1 is equal to 0.45430E+07
290
K1 is equal to 0.46294E+07
291
K1 is equal to 0.45788E+07
292
K1 is equal to 0.45431E+07
293
K1 is equal to 0.44857E+07
294
K1 is equal to 0.45208E+07
295
K1 is equal to 0.45376E+07
296
K1 is equal to 0.45449E+07
297
K1 is equal to 0.45363E+07
298
K1 is equal to 0.46534E+07
299
K1 is equal to 0.44606E+07
300
K1 is eq

K1 is equal to 0.63071E+07
534
K1 is equal to 0.61222E+07
535
K1 is equal to 0.64812E+07
536
K1 is equal to 0.66098E+07
537
K1 is equal to 0.64092E+07
538
K1 is equal to 0.62793E+07
539
K1 is equal to 0.66363E+07
540
K1 is equal to 0.69211E+07
541
K1 is equal to 0.63227E+07
542
K1 is equal to 0.67646E+07
543
K1 is equal to 0.65212E+07
544
K1 is equal to 0.69659E+07
545
K1 is equal to 0.68328E+07
546
K1 is equal to 0.69992E+07
547
K1 is equal to 0.67736E+07
548
K1 is equal to 0.66639E+07
549
K1 is equal to 0.67961E+07
550
K1 is equal to 0.67300E+07
551
K1 is equal to 0.67118E+07
552
K1 is equal to 0.66858E+07
553
K1 is equal to 0.67058E+07
554
K1 is equal to 0.67223E+07
555
K1 is equal to 0.74129E+07
556
K1 is equal to 0.72585E+07
557
K1 is equal to 0.73027E+07
558
K1 is equal to 0.74462E+07
559
K1 is equal to 0.72138E+07
560
K1 is equal to 0.73054E+07
561
K1 is equal to 0.73456E+07
562
K1 is equal to 0.71080E+07
563
K1 is equal to 0.70382E+07
564
K1 is equal to 0.73343E+07
565
K1 is eq

K1 is equal to 0.11181E+08
799
K1 is equal to 0.11250E+08
800
K1 is equal to 0.11305E+08
801
K1 is equal to 0.11386E+08
802
K1 is equal to 0.11186E+08
803
K1 is equal to 0.11150E+08
804
K1 is equal to 0.11672E+08
805
K1 is equal to 0.11267E+08
806
K1 is equal to 0.10921E+08
807
K1 is equal to 0.11150E+08
808
K1 is equal to 0.11166E+08
809
K1 is equal to 0.10925E+08
810
K1 is equal to 0.11730E+08
811
K1 is equal to 0.11600E+08
812
K1 is equal to 0.11797E+08
813
K1 is equal to 0.11721E+08
814
K1 is equal to 0.11839E+08
815
K1 is equal to 0.11666E+08
816
K1 is equal to 0.11990E+08
817
K1 is equal to 0.11900E+08
818
K1 is equal to 0.11992E+08
819
K1 is equal to 0.11860E+08
820
K1 is equal to 0.12158E+08
821
K1 is equal to 0.11709E+08
822
K1 is equal to 0.11831E+08
823
K1 is equal to 0.11989E+08
824
K1 is equal to 0.11680E+08
825
K1 is equal to 0.73611E+07
826
K1 is equal to 0.71437E+07
827
K1 is equal to 0.67396E+07
828
K1 is equal to 0.69581E+07
829
K1 is equal to 0.72137E+07
830
K1 is eq

K1 is equal to 0.10817E+08
1062
K1 is equal to 0.10767E+08
1063
K1 is equal to 0.11084E+08
1064
K1 is equal to 0.11270E+08
1065
K1 is equal to 0.11731E+08
1066
K1 is equal to 0.11384E+08
1067
K1 is equal to 0.11262E+08
1068
K1 is equal to 0.11528E+08
1069
K1 is equal to 0.11760E+08
1070
K1 is equal to 0.11434E+08
1071
K1 is equal to 0.11392E+08
1072
K1 is equal to 0.11309E+08
1073
K1 is equal to 0.11350E+08
1074
K1 is equal to 0.11584E+08
1075
K1 is equal to 0.11143E+08
1076
K1 is equal to 0.11732E+08
1077
K1 is equal to 0.11901E+08
1078
K1 is equal to 0.11705E+08
1079
K1 is equal to 0.11579E+08
1080
K1 is equal to 0.12422E+08
1081
K1 is equal to 0.12163E+08
1082
K1 is equal to 0.12025E+08
1083
K1 is equal to 0.12322E+08
1084
K1 is equal to 0.12145E+08
1085
K1 is equal to 0.12261E+08
1086
K1 is equal to 0.12138E+08
1087
K1 is equal to 0.12131E+08
1088
K1 is equal to 0.12258E+08
1089
K1 is equal to 0.12041E+08
1090
K1 is equal to 0.12110E+08
1091
K1 is equal to 0.12103E+08
1092
K1 is eq

...

K1 is equal to 0.13102E+08
1833
K1 is equal to 0.14203E+08
1834
K1 is equal to 0.13201E+08
1835
K1 is equal to 0.13442E+08
1836
K1 is equal to 0.13501E+08
1837
K1 is equal to 0.14201E+08
1838
K1 is equal to 0.12913E+08
1839
K1 is equal to 0.13374E+08
1840
K1 is equal to 0.13425E+08
1841
K1 is equal to 0.13267E+08
1842
K1 is equal to 0.12269E+08
1843
K1 is equal to 0.12412E+08
1844
K1 is equal to 0.13702E+08
1845
K1 is equal to 0.15160E+08
1846
K1 is equal to 0.15431E+08
1847
K1 is equal to 0.15743E+08
1848
K1 is equal to 0.15883E+08
1849
K1 is equal to 0.14598E+08
1850
K1 is equal to 0.14627E+08
1851
K1 is equal to 0.15193E+08
1852
K1 is equal to 0.14560E+08
1853
K1 is equal to 0.14717E+08
1854
K1 is equal to 0.14285E+08
1855
K1 is equal to 0.15123E+08
1856
K1 is equal to 0.15423E+08
1857
K1 is equal to 0.14955E+08
1858
K1 is equal to 0.14652E+08
1859
K1 is equal to 0.15485E+08
1860
K1 is equal to 0.16149E+08
1861
K1 is equal to 0.14753E+08
1862
K1 is equal to 0.15784E+08
1863
K1 is eq

K1 is equal to 0.21571E+08
2090
K1 is equal to 0.21817E+08
2091
K1 is equal to 0.22244E+08
2092
K1 is equal to 0.21098E+08
2093
K1 is equal to 0.21691E+08
2094
K1 is equal to 0.21773E+08
2095
K1 is equal to 0.20970E+08
2096
K1 is equal to 0.21224E+08
2097
K1 is equal to 0.21427E+08
2098
K1 is equal to 0.21607E+08
2099
K1 is equal to 0.22056E+08
2100
K1 is equal to 0.22746E+08
2101
K1 is equal to 0.22209E+08
2102
K1 is equal to 0.22877E+08
2103
K1 is equal to 0.22742E+08
2104
K1 is equal to 0.22715E+08
2105
K1 is equal to 0.23147E+08
2106
K1 is equal to 0.22894E+08
2107
K1 is equal to 0.22715E+08
2108
K1 is equal to 0.22429E+08
2109
K1 is equal to 0.22604E+08
2110
K1 is equal to 0.22688E+08
2111
K1 is equal to 0.22725E+08
2112
K1 is equal to 0.22681E+08
2113
K1 is equal to 0.23267E+08
2114
K1 is equal to 0.22303E+08
2115
K1 is equal to 0.24335E+08
2116
K1 is equal to 0.23791E+08
2117
K1 is equal to 0.25181E+08
2118
K1 is equal to 0.23959E+08
2119
K1 is equal to 0.24107E+08
2120
K1 is eq

K1 is equal to 0.18448E+08
2347
K1 is equal to 0.17680E+08
2348
K1 is equal to 0.17870E+08
2349
K1 is equal to 0.17346E+08
2350
K1 is equal to 0.18363E+08
2351
K1 is equal to 0.18728E+08
2352
K1 is equal to 0.18159E+08
2353
K1 is equal to 0.17791E+08
2354
K1 is equal to 0.18803E+08
2355
K1 is equal to 0.19610E+08
2356
K1 is equal to 0.17914E+08
2357
K1 is equal to 0.19166E+08
2358
K1 is equal to 0.18477E+08
2359
K1 is equal to 0.19737E+08
2360
K1 is equal to 0.19360E+08
2361
K1 is equal to 0.19831E+08
2362
K1 is equal to 0.19192E+08
2363
K1 is equal to 0.18881E+08
2364
K1 is equal to 0.19256E+08
2365
K1 is equal to 0.19068E+08
2366
K1 is equal to 0.19017E+08
2367
K1 is equal to 0.18943E+08
2368
K1 is equal to 0.19000E+08
2369
K1 is equal to 0.19047E+08
2370
K1 is equal to 0.21003E+08
2371
K1 is equal to 0.20566E+08
2372
K1 is equal to 0.20691E+08
2373
K1 is equal to 0.21098E+08
2374
K1 is equal to 0.20439E+08
2375
K1 is equal to 0.20699E+08
2376
K1 is equal to 0.20813E+08
2377
K1 is eq

K1 is equal to 0.26914E+08
2604
K1 is equal to 0.27125E+08
2605
K1 is equal to 0.27225E+08
2606
K1 is equal to 0.27269E+08
2607
K1 is equal to 0.27218E+08
2608
K1 is equal to 0.27920E+08
2609
K1 is equal to 0.26764E+08
2610
K1 is equal to 0.29202E+08
2611
K1 is equal to 0.28550E+08
2612
K1 is equal to 0.30217E+08
2613
K1 is equal to 0.28751E+08
2614
K1 is equal to 0.28928E+08
2615
K1 is equal to 0.29071E+08
2616
K1 is equal to 0.29277E+08
2617
K1 is equal to 0.28764E+08
2618
K1 is equal to 0.28672E+08
2619
K1 is equal to 0.30015E+08
2620
K1 is equal to 0.28971E+08
2621
K1 is equal to 0.28084E+08
2622
K1 is equal to 0.28672E+08
2623
K1 is equal to 0.28714E+08
2624
K1 is equal to 0.28093E+08
2625
K1 is equal to 0.30162E+08
2626
K1 is equal to 0.29828E+08
2627
K1 is equal to 0.30334E+08
2628
K1 is equal to 0.30139E+08
2629
K1 is equal to 0.30444E+08
2630
K1 is equal to 0.29999E+08
2631
K1 is equal to 0.30831E+08
2632
K1 is equal to 0.30601E+08
2633
K1 is equal to 0.30836E+08
2634
K1 is eq

K1 is equal to 0.22433E+08
2861
K1 is equal to 0.22373E+08
2862
K1 is equal to 0.22286E+08
2863
K1 is equal to 0.22353E+08
2864
K1 is equal to 0.22408E+08
2865
K1 is equal to 0.24710E+08
2866
K1 is equal to 0.24195E+08
2867
K1 is equal to 0.24342E+08
2868
K1 is equal to 0.24821E+08
2869
K1 is equal to 0.24046E+08
2870
K1 is equal to 0.24351E+08
2871
K1 is equal to 0.24485E+08
2872
K1 is equal to 0.23693E+08
2873
K1 is equal to 0.23461E+08
2874
K1 is equal to 0.24448E+08
2875
K1 is equal to 0.24379E+08
2876
K1 is equal to 0.24038E+08
2877
K1 is equal to 0.23926E+08
2878
K1 is equal to 0.24631E+08
2879
K1 is equal to 0.25044E+08
2880
K1 is equal to 0.26069E+08
2881
K1 is equal to 0.25298E+08
2882
K1 is equal to 0.25026E+08
2883
K1 is equal to 0.25618E+08
2884
K1 is equal to 0.26134E+08
2885
K1 is equal to 0.25410E+08
2886
K1 is equal to 0.25316E+08
2887
K1 is equal to 0.25132E+08
2888
K1 is equal to 0.25222E+08
2889
K1 is equal to 0.25742E+08
2890
K1 is equal to 0.24763E+08
2891
K1 is eq

In [42]:
# Saving the output data onto npy file
np.save('dataP1.npy',data)